## 팔머 펭귄
> X를 주면 펭귄을 분류하는 문제

* Iter1.
- SVC같은 선형 분류 모델 하나 선정
- 그리드 서치
- 파이프라인 

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("./data_pipeline/penguins_size.csv")
df.sample(15)

,species,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
150,Adelie,Dream,36.0,17.1,187.0,3700.0,FEMALE
317,Gentoo,Biscoe,46.9,14.6,222.0,4875.0,FEMALE
200,Chinstrap,Dream,51.5,18.7,187.0,3250.0,MALE
114,Adelie,Biscoe,39.6,20.7,191.0,3900.0,FEMALE
149,Adelie,Dream,37.8,18.1,193.0,3750.0,MALE
247,Gentoo,Biscoe,47.8,15.0,215.0,5650.0,MALE
159,Chinstrap,Dream,51.3,18.2,197.0,3750.0,MALE
232,Gentoo,Biscoe,45.5,13.7,214.0,4650.0,FEMALE
100,Adelie,Biscoe,35.0,17.9,192.0,3725.0,FEMALE
60,Adelie,Biscoe,35.7,16.9,185.0,3150.0,FEMALE


##### 문자로 저장된 데이터 확인

In [3]:
df['species'].unique()

array(['Adelie', 'Chinstrap', 'Gentoo'], dtype=object)

In [4]:
df['island'].unique()

array(['Torgersen', 'Biscoe', 'Dream'], dtype=object)

In [5]:
df['sex'].unique()

array(['MALE', 'FEMALE', nan, '.'], dtype=object)

In [6]:
df.isnull().sum()

species               0
island                0
culmen_length_mm      2
culmen_depth_mm       2
flipper_length_mm     2
body_mass_g           2
sex                  10
dtype: int64

In [7]:
penguins = df.dropna().isnull().sum()

In [8]:
penguins = df.dropna()

In [9]:
#인덱스가 끊기는 문제
penguins

,species,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,MALE
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,FEMALE
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,FEMALE
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,FEMALE
5,Adelie,Torgersen,39.3,20.6,190.0,3650.0,MALE
...,...,...,...,...,...,...,...
338,Gentoo,Biscoe,47.2,13.7,214.0,4925.0,FEMALE
340,Gentoo,Biscoe,46.8,14.3,215.0,4850.0,FEMALE
341,Gentoo,Biscoe,50.4,15.7,222.0,5750.0,MALE
342,Gentoo,Biscoe,45.2,14.8,212.0,5200.0,FEMALE


In [10]:
#인덱스를 새롭게 만들고 penguins에 그대로 저장
penguins.reset_index(drop=True, inplace=True)
penguins

,species,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,MALE
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,FEMALE
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,FEMALE
3,Adelie,Torgersen,36.7,19.3,193.0,3450.0,FEMALE
4,Adelie,Torgersen,39.3,20.6,190.0,3650.0,MALE
...,...,...,...,...,...,...,...
329,Gentoo,Biscoe,47.2,13.7,214.0,4925.0,FEMALE
330,Gentoo,Biscoe,46.8,14.3,215.0,4850.0,FEMALE
331,Gentoo,Biscoe,50.4,15.7,222.0,5750.0,MALE
332,Gentoo,Biscoe,45.2,14.8,212.0,5200.0,FEMALE


In [11]:
#성별에 .으로 저장된 데이터를 처리해야 함
penguins['sex'].unique()

array(['MALE', 'FEMALE', '.'], dtype=object)

In [12]:
penguins[penguins['sex'] == '.']

,species,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
327,Gentoo,Biscoe,44.5,15.7,217.0,4875.0,.


In [13]:
penguins.drop([327], inplace=True)

C:\Users\user\AppData\Local\Temp\ipykernel_15424\1472391410.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  penguins.drop([327], inplace=True)


In [14]:
#성별에 대한 전처리 완료
penguins['sex'].unique()

array(['MALE', 'FEMALE'], dtype=object)

In [15]:
#데이터를 카레고리로 만들기
penguins['species'] = penguins['species'].astype('category')

C:\Users\user\AppData\Local\Temp\ipykernel_15424\858310101.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  penguins['species'] = penguins['species'].astype('category')


In [16]:
#카테고리를 코드로 만들기
penguins['species_cod'] = penguins['species'].cat.codes

C:\Users\user\AppData\Local\Temp\ipykernel_15424\4164046270.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  penguins['species_cod'] = penguins['species'].cat.codes


In [17]:
#종별로 0~2의 코드가 부여됨
penguins

,species,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex,species_cod
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,MALE,0
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,FEMALE,0
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,FEMALE,0
3,Adelie,Torgersen,36.7,19.3,193.0,3450.0,FEMALE,0
4,Adelie,Torgersen,39.3,20.6,190.0,3650.0,MALE,0
...,...,...,...,...,...,...,...,...
329,Gentoo,Biscoe,47.2,13.7,214.0,4925.0,FEMALE,2
330,Gentoo,Biscoe,46.8,14.3,215.0,4850.0,FEMALE,2
331,Gentoo,Biscoe,50.4,15.7,222.0,5750.0,MALE,2
332,Gentoo,Biscoe,45.2,14.8,212.0,5200.0,FEMALE,2


In [18]:
penguins.info()

<class 'pandas.core.frame.DataFrame'>
Index: 333 entries, 0 to 333
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   species            333 non-null    category
 1   island             333 non-null    object  
 2   culmen_length_mm   333 non-null    float64 
 3   culmen_depth_mm    333 non-null    float64 
 4   flipper_length_mm  333 non-null    float64 
 5   body_mass_g        333 non-null    float64 
 6   sex                333 non-null    object  
 7   species_cod        333 non-null    int8    
dtypes: category(1), float64(4), int8(1), object(2)
memory usage: 19.0+ KB


In [19]:
penguins_feats = ['island', 'culmen_length_mm', 'culmen_depth_mm', 'flipper_length_mm', 'body_mass_g', 'sex']
penguins_labs = ['species_cod']

In [20]:
X, y = penguins[penguins_feats].values, penguins[penguins_labs].values

## SVC를 사용한 학습 및 평가

In [21]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.svm import SVC

from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix

In [22]:
#파이프라인 생성하기 (정의부)
n_feats = [1, 2 , 3, 4]
n_tras = Pipeline(steps=[('scaler', MinMaxScaler())])
c_feats = [0, 5]
c_tras = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])

In [23]:
#실행부
preprocessor  = ColumnTransformer(
    transformers = [
        ('num', n_tras, n_feats), #1, 2, 3, 4에 파이프라인을 실행
        ('cat', c_tras, c_feats)
    ]
)

In [24]:
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('svc', SVC(probability=True))
                           ])

In [25]:
SCV_scores = []
SVC_fimp = []
SVC_predictions = 0
FOLDS = 5

In [26]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [27]:
for fold, (train_idx, valid_idx) in enumerate(skf.split(penguins[penguins_feats].values,
                                                        penguins[penguins_labs].values)):
    print(f"Fold = {fold}")
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    X_val_train, X_val, y_val_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

    multi_model = pipeline.fit(X_val_train, y_val_train)
    preds_valid = multi_model.predict(X_val)
    acc = accuracy_score(y_val, preds_valid)
    SCV_scores.append(acc)

Fold = 0
Fold = 1
Fold = 2
Fold = 3
Fold = 4


c:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:1300: DataConversio

In [28]:
np.mean(SCV_scores)

0.9850746268656716

In [29]:
confusion_matrix(y_val, preds_valid)

array([[32,  0,  0],
       [ 1, 13,  0],
       [ 0,  0, 21]], dtype=int64)

In [30]:
preds_valid

array([2, 2, 1, 0, 0, 0, 0, 1, 1, 0, 2, 0, 1, 0, 0, 1, 0, 1, 0, 2, 1, 2,
       2, 2, 0, 0, 2, 2, 0, 2, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2, 1,
       0, 0, 2, 1, 0, 0, 1, 0, 1, 0, 0, 2, 2, 0, 2, 1, 0, 1, 2, 2, 2, 0,
       2], dtype=int8)